## Module 2 Homework

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
import re
import yfinance as yf
import time
from io import StringIO

In [2]:
# Helper functions
def scrape_table(url):
    """
    Scrape html tables from the given url.
    """
    # Read a web page
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    }
    response = requests.request("GET", url, headers=headers)

    # Parse scraping html to pandas DataFrame
    if response.status_code == 200:
        dfs = pd.read_html(StringIO(response.text))
    else:
        raise ValueError(f"Unable to scape the page!")
    
    return dfs


def get_price(price_range):
    """
    Get average price from the given price range.
    """
    if price_range is np.NaN:
        return np.NaN 

    price_pattern = "(\d+\.\d{2})"
    price_range_pattern = price_pattern + r"\s-\s" + price_pattern
    m = re.match(price_range_pattern, price_range)

    if not m:
        return float(price_range)

    return (float(m.group(1)) + float(m.group(2))) / 2


def calculate_daily_growth(tickers, ticker_category, days=7, start="2013-12-20", end="2024-01-01"):
    """
    Calculate 7D growth for the given ticker category.
    """
    ticker_dfs = []
    for ticker in tickers:
        ticker_df = yf.download(ticker, start=start, end=end, interval="1d")
    
        ticker_df = ticker_df.reset_index()
        ticker_df["ticker_category"] = ticker_category
        ticker_df[f"{days}d_growth"] = ticker_df["Adj Close"] / ticker_df["Adj Close"].shift(days)

        ticker_dfs.append(ticker_df)

        # Prevent overloading API server
        time.sleep(1)

    ticker_df = pd.concat(ticker_dfs, ignore_index=True)

    return ticker_df

### Question 1: IPO Filings Web Scraping and Data Processing

What's the total sum ($m) of 2023 filings that happend on Fridays?

In [3]:
# Get data
filings_df = scrape_table("https://stockanalysis.com/ipos/filings/")[0]

In [4]:
# Data preparation
filings_df["Price Range"] = filings_df["Price Range"].replace('-', np.NaN)
filings_df["Price Range"] = filings_df["Price Range"].str.replace('$', '')
filings_df["Shares Offered"] = filings_df["Shares Offered"].replace('-', np.NaN)

filings_df["Price"] = filings_df["Price Range"].apply(lambda x: get_price(x))

# Cast data type
filings_df["Filing Date"] = pd.to_datetime(filings_df["Filing Date"])
filings_df["Shares Offered"] = filings_df["Shares Offered"].astype(float)

# Create datetime features
filings_df["Year"] = filings_df["Filing Date"].dt.year
filings_df["Day of Week"] = filings_df["Filing Date"].dt.day_of_week

# Calculate total value of IPOs stocks
filings_df["Value"] = filings_df["Price"] * filings_df["Shares Offered"]

In [5]:
# Filter only filings in 2023 and on Friday
total_filings = round(filings_df[(filings_df["Year"] == 2023) & (filings_df["Day of Week"] == 4)]["Value"].sum() / 1e6)

print(f"Total sum of 2023 filings on Friday ($m): {total_filings}")

Total sum of 2023 filings on Friday ($m): 286


### Question 2: IPOs "Fixed days hold" strategy

Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Scenario: Let's assume that you managed to buy a new stock listed on IPOs on the first day at the Adj. Close price. Your strategy is to hold it for exactly X full days. For example, if X=1, you sell on the next day. Given around 200 IPO stocks, If you can pick top 25% stock (stocks in the 75% quantile growth), how many days you have to hold the stock in order to get the optimal return?

In [111]:
# Get data
years = ["2024", "2023"]
ipos_dfs = [scrape_table(f"https://stockanalysis.com/ipos/{year}")[0] for year in years] 
ipos_df = pd.concat(ipos_dfs, ignore_index=True)

In [112]:
# Data preparation
ipos_df["IPO Price"] = ipos_df["IPO Price"].str.replace('$', '')
ipos_df["Current"] = ipos_df["Current"].str.replace('$', '')
ipos_df["Return"] = ipos_df["Return"].str.replace('%', '')

# Cast data type
ipos_df["IPO Date"] = pd.to_datetime(ipos_df["IPO Date"], format="%b %d, %Y")

ipos_df["IPO Price"] = pd.to_numeric(ipos_df["IPO Price"])
ipos_df["Current"] = pd.to_numeric(ipos_df["Current"])
ipos_df["Return"] = pd.to_numeric(ipos_df["Return"]) / 100

# Filter only stocks IPOs before March, 2024 and ignore RYZB
ipos_df = ipos_df[(ipos_df["IPO Date"] < pd.to_datetime("2024-03-01")) & (ipos_df["Symbol"] != "RYZB")]

# Change PTHR to PTHRU according to Yahoo finance
ipos_df["Symbol"] = ipos_df["Symbol"].str.replace("PTHR", "PTHRU")

In [ ]:
# Get X-day future growth on IPOs date for each IPOs stock
symbols = ipos_df["Symbol"].tolist()

future_growths_dfs = []
for ticker in symbols:
    
    # Get daily OHLCV data; around 30 days from IPO Date
    start = ipos_df[ipos_df["Symbol"] == ticker]["IPO Date"].dt.strftime("%Y-%m-%d").values[0]
    end = (pd.to_datetime(start) + pd.Timedelta(days=31)).strftime("%Y-%m-%d")
    ticker_df = yf.download(tickers=ticker, start=start, interval="1d")[:31]

    # Calculate X-day future growth
    future_growths = {f"{i}d_future_growth": (ticker_df["Adj Close"].shift(-i) / ticker_df["Adj Close"]).iloc[0] for i in range(1, 31)}
    future_growths_df = pd.DataFrame(future_growths, index=[ticker])

    future_growths_dfs.append(future_growths_df)

    # Prevent overloading API server
    time.sleep(1)

future_growths_df = pd.concat(future_growths_dfs)

In [252]:
descriptive_df = future_growths_df.describe()
descriptive_df

,1d_future_growth,2d_future_growth,3d_future_growth,4d_future_growth,5d_future_growth,6d_future_growth,7d_future_growth,8d_future_growth,9d_future_growth,10d_future_growth,...,21d_future_growth,22d_future_growth,23d_future_growth,24d_future_growth,25d_future_growth,26d_future_growth,27d_future_growth,28d_future_growth,29d_future_growth,30d_future_growth
count,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,...,183.000000,183.000000,183.000000,183.000000,183.000000,182.000000,182.000000,182.000000,182.000000,182.000000
mean,0.951616,0.940686,0.935562,0.926667,0.914092,0.910672,0.900547,0.899755,0.904132,0.898005,...,0.920642,0.917850,0.917914,0.912733,0.907310,0.906325,0.892601,0.898485,0.895410,0.889593
std,0.162011,0.197966,0.240249,0.256661,0.288658,0.302250,0.292302,0.313816,0.345586,0.359648,...,0.505314,0.480825,0.493924,0.489273,0.492638,0.537474,0.502375,0.501480,0.480441,0.452594
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,...,0.048326,0.048326,0.049361,0.048326,0.048326,0.045219,0.044529,0.046945,0.042803,0.040387
25%,0.930659,0.898794,0.866908,0.855986,0.797155,0.799570,0.796129,0.774288,0.755546,0.712461,...,0.678704,0.622479,0.633110,0.641305,0.635149,0.600462,0.612186,0.646093,0.628665,0.623529
50%,1.000000,1.000000,0.998782,0.995516,0.993903,0.970152,0.974645,0.980808,0.985234,0.978063,...,0.984326,0.994409,0.991045,0.987043,0.972843,0.976420,0.981160,0.974952,0.974438,0.977535
75%,1.013844,1.018865,1.011578,1.013020,1.008295,1.006900,1.006871,1.006941,1.011545,1.007391,...,1.014209,1.035524,1.028015,1.035345,1.026972,1.028390,1.037564,1.040245,1.026362,1.023718
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,...,4.500000,3.871041,3.846154,3.803394,3.427273,4.817886,4.434146,4.570732,4.247155,3.629268


In [249]:
descriptive_df[descriptive_df.index == "75%"].T.sort_values(by="75%", ascending=False)

,75%
28d_future_growth,1.040245
27d_future_growth,1.037564
22d_future_growth,1.035524
24d_future_growth,1.035345
26d_future_growth,1.028390
23d_future_growth,1.028015
25d_future_growth,1.026972
29d_future_growth,1.026362
30d_future_growth,1.023718
14d_future_growth,1.019706


From this data, it suggests that you can not blindly pick IPOs stocks as the 50% (Median) on almost X-day future growth has the negative return. Therefore, this strategy may be a good option if you are confident that you can find top 25% stocks and interested in the maximum return is around 4%.

### Question 3: Is Growth Concentrated on the Largest Stocks?

Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks.

In [3]:
# List largest stocks
LARGEST_US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
LARGEST_EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
LARGEST_INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = LARGEST_US_STOCKS + LARGEST_EU_STOCKS + LARGEST_INDIA_STOCKS

In [4]:
# List large stocks
LARGE_US_STOCKS = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
LARGE_EU_STOCKS = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
LARGE_INDIA_STOCKS = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS'] 

LARGE_STOCKS = LARGE_US_STOCKS + LARGE_EU_STOCKS + LARGE_INDIA_STOCKS

In [ ]:
# Find 7D growth for each ticker category
largest_ticker_df = calculate_daily_growth(LARGEST_STOCKS, ticker_category="LARGEST")
large_ticker_df = calculate_daily_growth(LARGE_STOCKS, ticker_category="LARGE")

ticker_df = pd.concat([largest_ticker_df, large_ticker_df], ignore_index=True)

In [6]:
# Calcuate average daily growth
growth_df = ticker_df.groupby(by=["ticker_category", "Date"])["7d_growth"].mean().reset_index().dropna(ignore_index=True)

In [7]:
# Check performance of large stocks
performance_dfs = []
for g in growth_df.groupby("Date"):
    date = g[0]
    daily_growth_df = g[1]

    largest_daily_growth = daily_growth_df[daily_growth_df["ticker_category"] == "LARGEST"]["7d_growth"].values[0]
    large_daily_growth = daily_growth_df[daily_growth_df["ticker_category"] == "LARGE"]["7d_growth"].values[0] 

    performance_df = pd.DataFrame({
        "is_large_outperform": 1 if large_daily_growth > largest_daily_growth else 0
    }, index=[date])

    performance_dfs.append(performance_df)

performance_df = pd.concat(performance_dfs)

In [8]:
# Percentage when large stocks outperform largest stocks given the 10-year data
large_outperformed_percentage = round(performance_df.value_counts(normalize=True).iloc[1] * 100)

print(f"Given 10-year data, there is around {large_outperformed_percentage} of the times when large stocks outperform the largest stocks.")

Given 10-year data, there is around 47 of the times when large stocks outperform the largest stocks.


### Question 4: Trying Another Technical Indicators Strategy

What's the total gross profit ($k) you'll get from trading on CCI (Assume that there is no fees)?

In [84]:
# Get data
stocks_df = pd.read_parquet("data/stocks_df_combined_trunc_2014_2023.parquet.brotli")
cci_trading_df = stocks_df[["Ticker", "Date", "Weekday", "cci", "growth_future_5d"]].copy()

In [88]:
# Backtesting on CCI > 200 and trading only on Friday
cci_trading_df["is_invest"] = np.where(cci_trading_df["cci"] > 200, 1, 0)
investment_df = cci_trading_df[(cci_trading_df["is_invest"] == 1) & (cci_trading_df["Weekday"] == 4)].copy()

# Calculate gross profit
investment = 1000
investment_df["gross_profit"] = investment * investment_df["growth_future_5d"] - investment
total_gross_profit = investment_df["gross_profit"].sum()

print(f"Total Gross Profit ($k): {round(total_gross_profit / 1000)}")

Total Gross Profit ($k): 1


In [94]:
# If approximated fee is added; currency cost excluded
# https://www.degiro.ie/fees/calculator
fees = (460 + 460 + 2.5 + 138) / 1000

print(f"Total Fees ($k): {round(fees)}")

Total Fees ($k): 1


As you can see, there is no profit left after paying fees.

### [EXPLORATORY] Question 5: Finding Your Strategy for IPOs

You've seen in the first questions that the median and average investments are negative in IPOs, and you can't blindly invest in all deals.

How would you correct/refine the approach? Briefly describe the steps and the data you'll try to get (it should be generally feasible to do it from public sources - no access to internal data of companies)?

Ans: You can use industry type to segment and choose IPOs stocks to invest. For example, currently AI-related stocks should have the most promising return. Therefore, you can use select those IPOs stock that has AI-related business.